In [1]:
import os
import openai
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv, find_dotenv
import appointment_utils
import backup_utils
import firebase_utils

In [2]:
# Load environment variables
_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
def get_completion_from_messages(messages, model="gpt-4o-mini", temperature=0, max_tokens=250):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

In [4]:
def get_dates():
    today = datetime.today()
    tomorrow = today + timedelta(days=1)
    next_monday = today + timedelta(days=-today.weekday(), weeks=1)
    next_week = [next_monday + timedelta(days=i) for i in range(7)]
    return {
        "today": today.strftime('%Y-%m-%d'),
        "tomorrow": tomorrow.strftime('%Y-%m-%d'),
        "next_week": [day.strftime('%Y-%m-%d') for day in next_week]
    }

In [5]:
def get_date_from_day_name(day_name):
    today = datetime.today()
    days_of_week = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    target_day = days_of_week.index(day_name.lower())
    current_day = today.weekday()
    if target_day >= current_day:
        days_until_target = target_day - current_day
    else:
        days_until_target = 7 - (current_day - target_day)
    target_date = today + timedelta(days=days_until_target)
    return target_date.strftime('%Y-%m-%d')

In [6]:
# Load appointments from CSV
df = firebase_utils.load_appointments_from_csv()

# Save Appointments
def save_appointments(df):
    return firebase_utils.save_appointments_to_firebase(df), appointment_utils.save_appointments_local(df), backup_utils.create_backup()

In [7]:
def cancel_booking(df, name, date, email):
    df_filtered = df[(df['Name'] == name) & (df['Date'] == date) & (df['Email'] == email)]
    if df_filtered.empty:
        return df, False
    df = df.drop(df_filtered.index)
    return df, True

In [8]:
def check_availability(df, date, start, end):
    date = pd.to_datetime(date).strftime('%Y-%m-%d')
    start = pd.to_datetime(start, format='%H:%M:%S').strftime('%H:%M:%S')
    end = pd.to_datetime(end, format='%H:%M:%S').strftime('%H:%M:%S')
    
    # Filter the DataFrame for the specified date
    day_appointments = df[df['Date'] == date]
    
    # Check for overlapping appointments
    for _, row in day_appointments.iterrows():
        if (start < row['End'] and end > row['Start']):
            return False
    return True


In [9]:
def find_available_slots(date):
    date = pd.to_datetime(date).strftime('%Y-%m-%d')  # Ensure date is in the correct format
    
    working_hours = pd.date_range(start='09:00', end='18:00', freq='30min').time
    appointments = df[df['Date'] == date]
    
    booked_slots = [(row['Start'], row['End']) for _, row in appointments.iterrows()]

    available_slots = []
    slot_start = None

    for slot in working_hours:
        slot_str = slot.strftime('%H:%M:%S')
        is_booked = any(start <= slot_str < end for start, end in booked_slots)
        if not is_booked:
            if slot_start is None:
                slot_start = slot
        else:
            if slot_start is not None:
                available_slots.append(f"{slot_start.strftime('%I:%M %p')} - {slot.strftime('%I:%M %p')}")
                slot_start = None
    if slot_start is not None:
        available_slots.append(f"{slot_start.strftime('%I:%M %p')} - 06:00 PM")
    
    return available_slots

In [10]:
def handle_relative_time(relative_time):
    now = datetime.now()
    
    if relative_time.lower() == "later today":
        return now.strftime('%Y-%m-%d')
    
    elif relative_time.lower() == "tomorrow":
        tomorrow = now + timedelta(days=1)
        return tomorrow.strftime('%Y-%m-%d')
    
    elif relative_time.lower() == "next week":
        next_week = now + timedelta(weeks=1)
        return next_week.strftime('%Y-%m-%d')
    
    elif relative_time.lower().startswith("next"):
        days_of_week = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
        today_index = now.weekday()
        day_name = relative_time.lower().split("next ")[1]
        if day_name in days_of_week:
            target_index = days_of_week.index(day_name)
            delta_days = (target_index - today_index + 7) % 7
            target_date = now + timedelta(days=delta_days + 7)
            return target_date.strftime('%Y-%m-%d')
    
    else:
        # Handle specific day names
        days_of_week = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
        today_index = now.weekday()
        if relative_time.lower() in days_of_week:
            target_index = days_of_week.index(relative_time.lower())
            delta_days = (target_index - today_index + 7) % 7
            target_date = now + timedelta(days=delta_days)
            return target_date.strftime('%Y-%m-%d')
    
    # If no specific match, return today's date by default
    return now.strftime('%Y-%m-%d')

# Example usage
# print(handle_relative_time("later today"))  # Outputs today's date
# print(handle_relative_time("tomorrow"))  # Outputs tomorrow's date
# print(handle_relative_time("next week"))  # Outputs the same day next week
# print(handle_relative_time("next monday"))  # Outputs the date of next Monday
# print(handle_relative_time("Saturday"))  # Outputs the date of the upcoming Saturday


In [18]:
dates = get_dates()

system_message = f"""
You are Booking Appointment Agent, an automated service for booking appointments.
Your role is to assist customers with booking appointments, checking availability, and canceling bookings.

Follow these steps to assist customers:

1. **Greeting and Understanding the Query**:
   - Greet the customer warmly at the beginning.
   - Ask if they want to book an appointment, cancel a booking, or inquire about availability.
   - Remind the customer that bookings are only accepted from 9 AM to 6 PM, Monday to Saturday.
   - Understand whether the customer wants to book an appointment, cancel a booking, or make an inquiry.

2. **Checking Availability**:
   - If the customer asks for specific dates or relative times without giving a specific time (e.g., "later today", "tomorrow", "Saturday", "next Monday"):
     - Interpret the asked date or day into a specific date in the "Month Date, Year" or "%Y-%m-%d" format.
     - Use the `find_available_slots(date)` function with the interpreted date to find available slots on that day.
     - Inform the customer of the available slots on that day, specifying the day.
   - If the customer asks for specific dates with a specific time range:
     - Check its availability using the `check_availability(df, date, start, end)` function.
     - If the asked time is unavailable, inform the customer of available slots using the `find_available_slots(date)` function and ask if they want to change the time or date.
   - If the proposed date is fully booked, inform the customer and ask for an alternative date.
   - Accept various time formats (e.g., "4 to 6 pm", "from 4 PM to 6 PM", "2 to 5").
   - Assume times provided without AM or PM as PM for times between 1 to 6.

3. **Collecting Customer Details**:
   - Once a slot is confirmed, collect the customer's name, email, and contact number.
   - Ask for any missing information if needed.

4. **Confirmation and Data Entry**:
   - Confirm all details (name, date, time, email, contact number) with the customer, specifying the day of the week.
   - After the customer confirms, verify availability using `check_availability(df, date, start, end)` function once more.
   - If the slot is available, save the booking data using the `save_appointments(df)` function.
   - Inform the customer that the appointment has been booked.

5. **Booking Cancellation**:
   - Ask for the customer's name and the date of the booking they want to cancel.
   - Ask for the customer's email.
   - Verify the provided data with the database using the `cancel_booking(df, name, date, email)` function.
   - Confirm the cancellation with the customer.
   - Delete the booking from the database using the `cancel_booking(df, name, date, email)` function.
   - Save the latest appointments data using the `save_appointments(df)` function.
   - Notify the customer whether the cancellation was successful or not.

6. **Date Management**:
   - Today is {dates['today']}.
   - Tomorrow is {dates['tomorrow']}.
   - The dates for the next week (starting from Monday) are: {dates['next_week']}.

7. **Error Handling**:
   - Handle invalid dates/times and incomplete data gracefully.
   - Ask the customer for necessary information if any details are missing.

8. **Maintaining Conversation Context**:
   - Maintain the conversation context to avoid repeated information requests.
   - Continue from the previous conversation smoothly.

9. **Data Privacy**:
   - Never share another customer's data with the user.

Respond in a concise, friendly, and conversational style to ensure a smooth and pleasant booking experience for the customer.
"""


In [13]:
def call_assistant_function(intent, user_data):
    global df  # Ensure df is accessible within the function

    if intent == 'check_availability':
        date = user_data.get('date')
        if not date:
            # Handle relative time to get the date
            date = handle_relative_time(user_data.get('relative_time'))
        
        available_slots = find_available_slots(date)
        return f"Available slots for {date}: {available_slots}"
    
    elif intent == 'book_appointment':
        date = user_data.get('date')
        start = user_data.get('start')
        end = user_data.get('end')
        
        # Check availability
        if not check_availability(df, date, start, end):
            available_slots = find_available_slots(date)
            return f"Sorry, the slot from {start} to {end} on {date} is not available. Available slots: {available_slots}. Would you like to book one of these?"
        else:
            # Proceed with booking
            name = user_data.get('name')
            email = user_data.get('email')
            contact_number = user_data.get('contact_number')
            booking_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            new_appointment = {
                'Name': name,
                'Date': date,
                'Start': start,
                'End': end,
                'Email': email,
                'Contact Number': contact_number,
                'Booking Time': booking_time
            }
            
            # Update and save appointments
            df = appointment_utils.update_appointments(df, new_appointment)
            save_appointments(df)
            
            return f"Booking confirmed for {name} on {date} from {start} to {end}."

    elif intent == 'cancel_appointment':
        name = user_data.get('name')
        date = user_data.get('date')
        email = user_data.get('email')
        
        df, success = cancel_booking(df, name, date, email)
        
        if success:
            save_appointments(df)
            return f"Booking for {name} on {date} has been successfully cancelled."
        else:
            return f"No booking found for {name} on {date} with the email {email}."

    else:
        return "Sorry, I didn't understand your request."


In [14]:
# Example test
user_message = "do you have any slots tomorrow?"

messages =  [  
    {'role': 'system', 'content': system_message},    
    {'role': 'user', 'content': user_message},
    {'role': 'assistant', 'content': call_assistant_function('check_availability', {'relative_time': 'tomorrow'})},
] 

response = get_completion_from_messages(messages)
print(response)


We have available slots tomorrow, July 28th, from 9:00 AM to 6:00 PM. Would you like to book a specific time within that range?


In [16]:
user_message = "I would like to book an appointment for Thursday from 3 PM to 5 PM."

messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': user_message},
    {'role': 'assistant', 'content': call_assistant_function('book_appointment', {
        'date': '2024-08-01', 'start': '15:00:00', 'end': '17:00:00', 
        'name': 'Ray', 'email': 'ray@example.com', 'contact_number': '1234567890'
    })},
]

response = get_completion_from_messages(messages)
print(response)


Sorry, the slot from 3 PM to 5 PM on Thursday, August 1st, is not available. However, we do have the following available slots on that day:

- 9:00 AM - 3:00 PM
- 5:00 PM - 6:00 PM

Would you like to book one of these slots instead?


In [17]:
user_message = "I would like to cancel my appointment on 2024-08-01."

messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': user_message},
    {'role': 'assistant', 'content': call_assistant_function('cancel_appointment', {
        'name': 'Ray', 'date': '2024-08-01', 'email': 'ray@example.com'
    })},
]

response = get_completion_from_messages(messages)
print(response)


I can help you with that! Could you please provide me with your name and the email associated with your appointment?
